<a href="https://colab.research.google.com/github/ProValarous/Reinforcement-Learning-Labs/blob/main/Rec_module_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recitation Module #2: Dynammic Programming & Monte Carlo Methods

Topics covered:
- Value Iteration
- Policy Interation
- Monte-Carlo Prediction


## Importing Libraries and Dependencies


In [ ]:
# HIDE OUTPUT
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

In [ ]:
# HIDE OUTPUT
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!sudo apt-get install xvfb
!pip install xvfbwrapper
!pip install gymnasium

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.14).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
%matplotlib inline
import gym
import gymnasium as gym
import numpy as np
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from matplotlib import pyplot as plt
import base64
import glob
from IPython.display import HTML, display
from collections import defaultdict

In [ ]:
## Some Display Utility functions

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""


def show_video():
    import base64
    import glob
    from IPython.display import HTML, display as ipythondisplay

    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = open(mp4, 'rb').read()
        encoded = base64.b64encode(video).decode('ascii')
        display(HTML(data=f'''
            <video width="640" height="480" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
            </video>
        '''))
    else:
        print("Could not find video")


def wrap_env(env):
    env = RecordVideo(env, './video')
    return env


### Techniques for solving MDPs can be separated into three categories:

1. **Value-based techniques:** aim to learn the value of states (or learn an estimate for value of states) and
actions: that is, they learn value functions or Q functions. We then use policy extraction to get a
policy for deciding actions.
2. **Policy-based techniques:** learn a policy directly, which completely by-passes learning values of states
or actions all together. This is important if for example, the state space or the action space are massive
or infinite. If the action space is infinite, then using policy extraction as defined in Part I is not
possible because we must iterate over all actions to find the optimal one. If we learn the policy
directly, we do not need this.
3. **Hybrid techniques:** that combine value- and policy-based techniques

## Value Iteration

Value Iteration is a dynamic-programming method for finding the optimal value function $V^*$ by solving the
Bellman equations iteratively. It uses the concept of dynamic programming to maintain a value function $V^*$
that approximates the optimal value function $V^*$ , iteratively improving $V$ until it converges to $V^*$ (or
close to it).



**The algorithm is as follows:**

**Input:** MDP $M = \langle S, s_0, A, P_a(s' \mid s), r(s, a, s') \rangle$  
**Output:** Value function $V$  

Set $V$ to arbitrary value function; e.g., $V(s) = 0$ for all $s$  

---

**Repeat**  
- $\Delta \gets 0$  
- **For each** $s \in S$  
  - $V'(s) \gets \max_{a \in A(s)} \sum_{s' \in S} P_a(s' \mid s) \left[ r(s, a, s') + \gamma V(s') \right]$ ← _Bellman equation_  
  - $\Delta \gets \max(\Delta, |V'(s) - V(s)|)$  
- $V \gets V'$  

**Until** $\Delta \leq \theta$  

---



We could also write the algorithm using the idea of Q-values, which is closer to a code-based
implementation:

<img src="https://lcalem.github.io/imgs/sutton/value_iteration.png" alt="Value Iteration" width="700" height="300">


In [ ]:
def value_iteration(env, gamma=1.0):
    import numpy as np

    value_table = np.zeros(env.observation_space.n)

    no_of_iterations = 100000
    threshold = 1e-20

    for i in range(no_of_iterations):
        updated_value_table = np.copy(value_table)

        for state in range(env.observation_space.n):
            Q_values = []

            for action in range(env.action_space.n):
                q = 0
                for prob, next_state, reward, done in env.P[state][action]:
                    q += prob * (reward + gamma * value_table[next_state])
                Q_values.append(q)

            updated_value_table[state] = max(Q_values)

        # Check for convergence
        if np.sum(np.fabs(updated_value_table - value_table)) <= threshold:
            print(f"Value iteration converged at iteration {i}")
            break

        value_table = updated_value_table

    return value_table


In [ ]:
def extract_policy(value_table, gamma=1.0):
    policy = np.zeros(env.observation_space.n)

    for state in range(env.observation_space.n):
        Q_values = []

        for action in range(env.action_space.n):
            q = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q += prob * (reward + gamma * value_table[next_state])
            Q_values.append(q)

        # FIX: ensure policy has integer values
        policy[state] = int(np.argmax(Q_values))

    return policy


In [ ]:
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

# Try generating a valid solvable map
while True:
    custom_map = generate_random_map(size=8)
    env_train = gym.make('FrozenLake-v1', desc=custom_map, render_mode="rgb_array")
    env = env_train.unwrapped

    # Test if goal is reachable with a random policy
    test_policy = lambda s: env.action_space.sample()
    state, _ = env.reset()
    reached = False
    for _ in range(200):
        action = test_policy(state)
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        if reward == 1.0:
            reached = True
            break
        if done:
            break

    if reached:
        break  # keep this env_train


In [ ]:
optimal_value_function = value_iteration(env, gamma=0.9)


Value iteration converged at iteration 282


In [ ]:
optimal_policy = extract_policy(optimal_value_function, gamma=0.9)


In [ ]:
print(optimal_policy)

[3. 2. 2. 3. 2. 2. 2. 2. 0. 2. 0. 0. 2. 2. 2. 1. 1. 1. 0. 0. 0. 2. 2. 1.
 1. 1. 0. 0. 1. 3. 2. 1. 1. 1. 1. 0. 0. 0. 2. 1. 3. 3. 3. 3. 1. 1. 1. 1.
 0. 0. 0. 0. 2. 2. 1. 1. 0. 0. 1. 1. 2. 2. 2. 0.]


In [ ]:
# Wrap the environment to enable video recording
env = wrap_env(env_train)

# Reset the environment
state, _ = env.reset()

# Start video recording
env.start_video_recorder()

next_state = state

# Follow the optimal policy
for _ in range(10000):
    action = int(optimal_policy[next_state])
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    if done:
        break

# Close the video recorder and the environment
env.close_video_recorder()
env.close()


/usr/local/lib/python3.11/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/video/rl-video-episode-0.mp4.
Moviepy - Writing video /content/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/rl-video-episode-0.mp4
Moviepy - Building video /content/video/rl-video-episode-0.mp4.
Moviepy - Writing video /content/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/rl-video-episode-0.mp4


In [ ]:
# Display Output
show_video()

## **Policy Iteration**

Policy Iteration is a fundamental method in **Dynamic Programming (DP)** to solve **Markov Decision Processes (MDPs)**. It helps find an optimal policy $ \pi^* $ that maximizes the expected cumulative reward.

---

### **Key Idea**  
Policy Iteration alternates between two steps:

1. **Policy Evaluation** – Compute the value function $ V^\pi(s) $ for a given policy $ \pi $.
2. **Policy Improvement** – Update the policy by selecting actions that maximize the expected reward.

This repeats until the policy converges to the optimal policy.

---

### **Steps of Policy Iteration**
1. **Initialize** a random policy $ \pi_0 $.
----
2. **Policy Evaluation:**  
   - Compute the value function $ V^\pi(s) $ by solving the Bellman equation: $$ V^\pi(s) = \sum_{s'} P(s' | s, \pi(s)) \left[ R(s, \pi(s), s') + \gamma V^\pi(s') \right] $$
   - This finds the expected reward for following $ \pi $.

**Here's a more algorithmic approach:**

**Input:** $\pi$ the policy for evaluation, $V^\pi$ value function, and  
MDP $M = \langle S, s_0, A, P_a(s' \mid s), r(s, a, s') \rangle$  

**Output:** Value function $V^\pi$  

**Repeat**  
- $\Delta \gets 0$  
- **For each** $s \in S$  
  - $$V'^\pi(s) \gets \sum_{s' \in S} P_{\pi(s)}(s' \mid s) \left[ r(s, a, s') + \gamma V^\pi(s') \right]$$  
    _Policy evaluation equation_  
  - $\Delta \gets \max(\Delta, |V'^\pi(s) - V^\pi(s)|)$  
- $V^\pi \gets V'^\pi$  

**Until** $\Delta \leq \theta$  

-----

3. **Policy Improvement:**  
   - For each state $ s $, update the policy:$$ \pi(s) = \arg\max_a \sum_{s'} P(s' | s, a) \left[ R(s, a, s') + \gamma V^\pi(s') \right]$$
   - If the policy doesn't change, stop. Otherwise, repeat from Step 2.

---
**Here's a full algorithmic Policy Iteration:**


**Input:** MDP $M = \langle S, s_0, A, P_a(s' \mid s), r(s, a, s') \rangle$  
**Output:** Policy $\pi$  

Set $V^\pi$ to arbitrary value function; e.g., $V^\pi(s) = 0$ for all $s$  
Set $\pi$ to arbitrary policy; e.g., $\pi(s) = a$ for all $s$, where $a \in A$ is an arbitrary action  

**Repeat**  
- Compute $V^\pi(s)$ for all $s$ *(using policy evaluation)*  
- **For each** $s \in S$:  
  - $$\pi(s) \gets \arg\max_{a \in A(s)} Q^\pi(s, a)$$  
- **Until** $\pi$ does not change *(policy Improvement)*

---

### **Example: Grid World**
Consider a 3×3 grid where an agent moves **up, down, left, or right** to reach a goal while receiving rewards.

Policy Iteration helps the agent learn the best moves by:
- Evaluating the current policy.
- Updating the policy to improve its actions.
- Repeating until the policy stops changing.

---

### **Comparison with Value Iteration**
| **Method**          | **Approach** |
|---------------------|-------------|
| **Policy Iteration** | Alternates between policy evaluation and improvement. |
| **Value Iteration**  | Updates values directly using the Bellman optimality equation. |

Both find the optimal policy, but **Value Iteration** can be more efficient for large state spaces.

---


In [ ]:
def compute_value_function(policy):
    num_iterations = 1000
    threshold = 1e-20
    gamma = 1.0

    value_table = np.zeros(env.observation_space.n)

    for i in range(num_iterations):
        updated_value_table = np.copy(value_table)

        for s in range(env.observation_space.n):
            a = int(policy[s])  # make sure it's an int (policy might be float)
            v = 0
            for prob, next_state, reward, done in env.P[s][a]:
                v += prob * (reward + gamma * value_table[next_state])
            updated_value_table[s] = v

        # Convergence check
        if np.sum(np.abs(updated_value_table - value_table)) <= threshold:
            print(f"Policy evaluation converged at iteration {i}")
            break

        value_table = updated_value_table

    return value_table


In [ ]:
def policy_iteration(env):
    num_iterations = 1000
    policy = np.zeros(env.observation_space.n)

    for i in range(num_iterations):
        # Step 1: Policy Evaluation
        value_function = compute_value_function(policy)

        # Step 2: Policy Improvement
        new_policy = extract_policy(value_function)

        # Step 3: Check if the policy has converged
        if np.array_equal(policy, new_policy):
            print(f"Policy iteration converged at iteration {i}")
            break

        # Step 4: Update the policy
        policy = new_policy

    return policy


In [ ]:

env = env_train.unwrapped


In [ ]:
optimal_policy = policy_iteration(env)


Policy evaluation converged at iteration 93
Policy evaluation converged at iteration 707
Policy evaluation converged at iteration 793
Policy evaluation converged at iteration 798
Policy evaluation converged at iteration 727
Policy evaluation converged at iteration 919


In [ ]:
env = wrap_env(env_train)
state, _ = env.reset()
next_state = state

for _ in range(10000):
    action = int(optimal_policy[next_state])
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        break

env.close_video_recorder()
env.close()


NameError: name 'video_path' is not defined

In [ ]:
# Display Output
show_video()

## What is Monte Carlo?

Monte Carlo methods are a class of algorithms that rely on repeated random sampling to obtain numerical results. The core idea is to use randomness to solve problems that might be deterministic in principle. In the context of Reinforcement Learning (RL), Monte Carlo methods are used to estimate the value of states or state-action pairs by running multiple episodes and observing the outcomes. The key concept here is **trial and error**—we perform random experiments (trials), and based on the results, we compute averages that approximate the true values.

#### Key Points:
- **Random sampling**: Monte Carlo methods use randomness to simulate and explore possible outcomes.
- **Episodic tasks**: They are often used in episodic environments where each episode ends in a terminal state.
- **No need for a model**: Monte Carlo doesn’t require a complete model of the environment, unlike some other methods in RL, such as Dynamic Programming.

### Where is it Applicable in Reinforcement Learning?

In RL, Monte Carlo methods are used to solve Markov Decision Processes (MDPs) by estimating the value functions based on actual experiences, which is different from Dynamic Programming (DP) methods like Policy Iteration or Value Iteration where we know the model dynamics.

- **Monte Carlo**:
    - **Model-free**: It doesn’t need the transition probabilities or reward model of the environment.
    - **Episodic**: It requires full episodes to compute an average return.
    - **Learning from interaction**: Monte Carlo learns from real or simulated experiences, by averaging returns for visited states.
    - **Convergence**: It only converges after multiple episodes, especially for larger MDPs.
  
- **Dynamic Programming**:
    - **Model-based**: DP requires a known model of the environment, including transition probabilities and rewards.
    - **Bootstrapping**: DP methods use current estimates of value functions to update state values iteratively.
    - **Complete information**: It works by sweeping over the entire state space and updating values simultaneously.

Monte Carlo methods are more useful when we can’t directly model the environment, whereas Dynamic Programming is effective when we know the full MDP model. Monte Carlo works well for large-scale problems or when interacting with the environment is cheap or necessary.


### Estimating pi-value using Monte Carlo

Monte Carlo methods are often applied to estimate values that are otherwise hard to calculate, such as the mathematical constant π (pi). Here’s how we can estimate π using Monte Carlo:

- **Basic Idea**: Imagine a unit square with a circle inscribed in it (diameter = 1). The area of the square is 1, and the area of the circle is π/4. If we randomly generate points within the square, the ratio of points that land inside the circle to the total number of points should approximate the area ratio between the circle and square, which is π/4.

- **Steps**:
  1. Randomly generate points $(x, y)$ in the unit square.
  2. Check if the point lies inside the circle i.e., if $( x^2 + y^2 \leq 1 )$.
  3. We calculate the value π (pi) by multiplying four to the division of the number of points inside the circle to the number of points inside the square.
  4. If we increase the number of samples, the better we can approximate.

This is a simple but powerful demonstration of Monte Carlo’s ability to estimate quantities through random sampling.

<center>
<img src="https://help.ovhcloud.com/public_cloud-data_analytics-data_processing-40_tutorial_calculate_pi-images-monte_carlo_graph.png" alt="pi_est" width="200" height="200">
</center>

In [ ]:
### TODO: Estimate the value of pi using Monte Carlo

import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

def pi_check(estimate): # DO NOT MODIFY THIS FUNCTION
    pi = math.pi
    difference = abs(pi - estimate)
    if difference < 0.00001:
        print('Test Case passed')
    else:
        print('Test Case not passed')
def mc_pi_estimator(num_samples=100000):
    inside_circle = 0

    for _ in range(num_samples):
        x = np.random.uniform(0, 1)
        y = np.random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1

    estimate = 4 * (inside_circle / num_samples)
    return estimate



### Making a Monte Carlo Integrator

Monte Carlo methods are widely used to approximate integrals, especially for high-dimensional or complex integrals where traditional methods fail or are inefficient.

- **Problem**: You want to compute the integral of a function  $f(x)$ over an interval $[a, b]$.
  
- **Monte Carlo Approach**:
  1. **Random Sampling**: Generate random points $ x_i $ uniformly in the interval $[a, b]$.
  2. **Function Evaluation**: Compute the function value $ f(x_i) $ for each sampled point.
  3. **Averaging**: Compute the average of the function values and scale it by the interval width, $( b - a )$.

The estimated integral is:
$$
\int_a^b f(x) dx \approx (b - a) \times \frac{1}{N} \sum_{i=1}^{N} f(x_i)
$$
where $( N )$ is the number of samples.

This approach is useful when the function is difficult to integrate analytically, or when the domain of integration is high-dimensional. Monte Carlo integration's accuracy improves as the number of samples increases, though convergence is slow compared to other numerical methods.


In [ ]:

def mc_integrator(f, a, b, num_samples=10000):
    """
    Monte Carlo Integration of function f over [a, b]
    """
    samples = np.random.uniform(a, b, num_samples)
    values = f(samples)
    estimate = (b - a) * np.mean(values)
    return estimate


Solve following Integral using your implemented MC Integrator:

$$
    \int_0^{\frac{\pi}{2}} \frac{\sqrt{sin(x)}}{\sqrt{sin(x)}+\sqrt{cos(x)}}
$$

In [ ]:
def f(x):
    return np.sqrt(np.sin(x)) / (np.sqrt(np.sin(x)) + np.sqrt(np.cos(x)))

In [ ]:
# Integration limits
a = 0
b = np.pi / 2

# Estimate the integral
estimate = mc_integrator(f, a, b, num_samples=100000)
print("Estimated value of the integral:", estimate)


Estimated value of the integral: 0.7844989625153846


## Monte Carlo Prediction

### General Overview

Monte Carlo Prediction is a technique in Reinforcement Learning (RL) used to estimate the value of states (or state-action pairs) based on the observed returns from sampled episodes. The goal is to approximate the value function, which represents the expected return (cumulative future rewards) starting from a given state under a specific policy.

- **Objective**: For a given policy $ \pi $, Monte Carlo methods aim to estimate the **value function** $ V^\pi(s) $ for each state $ s $, which is the expected return starting from $ s $, and following the policy $ \pi $. We estimate the expected return using mean returns.
- **Episode-based learning**: Monte Carlo methods compute value estimates by averaging returns across multiple episodes. An episode is a sequence of states, actions, and rewards that ends in a terminal state.
  
In Monte Carlo Prediction, the value of a state is updated based on the actual returns received in episodes, which allows it to handle non-Markov environments and environments where a model of the dynamics is unknown. This is why Monte Carlo prediction is called *'model-free'* learning algorithm. This method works particularly well for **episodic tasks** where we can easily define when an episode starts and ends.

---

The steps involved in Monte Carlo prediction are as follows:

1. First, we initialize a random value to our value function.
2. Then we initialize an emplty list called a return to store our return.
3. Then for each state in the episode, we calculate the return.
4. Next, we append the return to out return list.
5. Finally, we take average of return as our value function.

----

The Monte Carlo Prediction algorithm is of two types;

- First Visit Monte Carlo
- Every Visit Monte Carlo

### First Visit Monte Carlo

In First Visit Monte Carlo (MC) we average the return only the first time the state is visited in an episode. In Reinforcement Learning, a state may be visited multiple times within a single episode.
- First Visit Monte Carlo focuses on the first occurrence of each state to provide an unbiased estimate of its value.

- First Visit ensures that the value of a state is only updated once per episode, based on the first time it was encountered.
- This method helps avoid correlation between multiple visits to the same state in a single episode, which can introduce bias into the value estimation.
By averaging the returns for the first visits over many episodes, we get a robust estimate of the value function for each state.

#### Algorithm for First Visit MC Prediction:

1. **Initialize** the value function $ V(s) $ for all states arbitrarily (or to zeros) and a counter $ N(s) = 0 $ for each state $ s $ to track the number of first visits to $ s $.

2. **Simulate episodes**: For each episode:
    - Generate a complete episode by following the given policy $ \pi $.
    - An episode is a sequence of states, actions, and rewards:
      $$
      (S_0, A_0, R_1, S_1, A_1, R_2, \dots, S_T)
      $$
      where $ S_T $ is the terminal state, and $ T $ is the time step at which the episode ends.

3. **Track first visits**: For each state $ S_t $ visited in the episode, check if this is the first time it has been visited within the episode. If so:
    - **Compute the return** $ G_t $, which is the total discounted reward from time step $ t $ until the end of the episode:
      $$
      G_t = R_{t+1} + \gamma R_{t+2} + \dots + \gamma^{T-t} R_T
      $$
    - **Update the value function** $ V(S_t) $ based on this return:
      $$
      V(S_t) \leftarrow V(S_t) + \alpha [G_t - V(S_t)]
      $$
      where $ \alpha $ is the learning rate, or:
      $$
      α = \frac{1}{N(S_t)}
      $$
      if you are averaging over all returns for state $ S_t $.
  
    - **Update the count** for the first visit to the state:
      $$
      N(S_t) \leftarrow N(S_t) + 1
      $$

4. **Repeat** this process over many episodes to improve the estimates of the value function.

---

More concisely, the algorithm is as follows:

<img src="https://lcalem.github.io/imgs/sutton/first_visit.png" alt="Girl in a jacket" width="700" height="300">


In [ ]:
def generate_episodes(env, policy, max_steps=1000):
    episode = []
    state, _ = env.reset()

    for _ in range(max_steps):
        action = int(policy(state))  # For functional/random policy
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode.append((state, action, reward))

        if done:
            break
        state = next_state

    return episode


In [ ]:
from collections import defaultdict


def first_visit_mc_prediction(env, policy, n_episodes=5000, gamma=1.0):
    value_table = defaultdict(float)
    N = defaultdict(int)

    for _ in range(n_episodes):
        episode = generate_episodes(env, policy)

        G = 0
        visited = set()

        for t in range(len(episode) - 1, -1, -1):
            state, _, reward = episode[t]
            G = gamma * G + reward

            if state not in visited:
                visited.add(state)
                N[state] += 1
                value_table[state] += (G - value_table[state]) / N[state]

    return value_table


### Every Visit Monte Carlo

**Every Visit Monte Carlo** is a model-free prediction method in Reinforcement Learning (RL) used to estimate the value function $ V^\pi(s) $ of a given policy $ \pi $. Like other Monte Carlo methods, it uses complete episodes of experience and updates the estimated value of a state based on the **returns** from multiple episodes.

The key feature of **Every Visit Monte Carlo** is that it updates the value of a state every time the state is visited within an episode, rather than just the first time (which is what happens in First Visit Monte Carlo).

---

### Why Use Every Visit Monte Carlo?

In environments where states can be revisited multiple times within the same episode, **Every Visit Monte Carlo** provides a way to leverage all available information by updating the value of a state each time it is visited.

- **More frequent updates**: By updating the value of a state for every visit, you can learn faster in environments where the agent frequently revisits the same state.
- **Utilizes more data**: Every occurrence of a state gives more opportunities to estimate the value, which may lead to faster convergence of the value function compared to First Visit Monte Carlo.

---

In [ ]:
def every_visit_mc_prediction(env, policy, n_episodes=5000, gamma=1.0):
    value_table = defaultdict(float)
    N = defaultdict(int)

    for _ in range(n_episodes):
        episode = generate_episodes(env, policy)

        G = 0

        for t in range(len(episode) - 1, -1, -1):
            state, _, reward = episode[t]
            G = gamma * G + reward
            N[state] += 1
            value_table[state] += (G - value_table[state]) / N[state]

    return value_table



### Key Differences between First Visit Monte Carlo and Every Visit Monte Carlo:

- **Every Visit Monte Carlo** updates the value of a state every time it is visited within an episode, whereas **First Visit Monte Carlo** updates the value only the first time a state is encountered.
- **Every Visit Monte Carlo** tends to use more data for updating the value function since it uses all occurrences of each state within an episode, potentially leading to faster learning in some cases.
- **First Visit Monte Carlo** avoids correlation between multiple visits to the same state, while Every Visit Monte Carlo exploits multiple visits to gather more data but may introduce more noise or bias in certain environments.

Both methods will converge to the correct value function given enough episodes, but they may differ in how fast they converge depending on the environment and the task.

### Playing Black-Jack with Monte Carlo Methods


In [ ]:
from collections import defaultdict

def extract_policy(env, value_table, gamma=1.0):
    policy = defaultdict(int)

    for state in value_table.keys():
        Q_table = np.zeros(env.action_space.n)

        for action in range(env.action_space.n):
            for trans_prob, next_state, reward, _ in env.P[state][action]:
                Q_table[action] += trans_prob * (reward + gamma * value_table[next_state])

        policy[state] = int(np.argmax(Q_table))

    return policy


In [ ]:
import gym

env_train = gym.make('Blackjack-v1', natural=False, sab=False)



In [ ]:
env = env_train.unwrapped

# Random policy as function
random_policy = lambda state: env.action_space.sample()

# First-Visit Monte Carlo to estimate value of states under random policy
value_first_visit_mc = first_visit_mc_prediction(env, policy=random_policy, n_episodes=500000)
# Note: We do not call extract_policy() for Blackjack because it's a model-free environment.
# The transition dynamics env.P are not accessible, so policy extraction via Bellman backup isn't possible.
# Instead, we use the estimated value_table directly or proceed to Monte Carlo Control to derive a better policy.



In [ ]:
# Using the same random policy for every-visit MC
value_every_visit_mc = every_visit_mc_prediction(env, policy=random_policy, n_episodes=500000)

# We do not extract policy from the value table in Blackjack, since it's model-free


In [ ]:
import gym
from gym.wrappers import RecordVideo
import numpy as np

# Patch for compatibility: fix 'np.bool8' error
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

# Create environment with video recording
env = gym.make('Blackjack-v1', natural=False, sab=False, render_mode="rgb_array")
env = RecordVideo(env, video_folder="./video", episode_trigger=lambda ep: True)

# Reset environment
state, _ = env.reset()
next_state = state

# Run episode with random policy
for _ in range(10000):
    action = int(random_policy(next_state))
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        break

env.close_video_recorder()
env.close()


Moviepy - Building video /content/video/rl-video-episode-0.mp4.
Moviepy - Writing video /content/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/rl-video-episode-0.mp4


In [ ]:
# Display Output
show_video()

In [ ]:
print("First-Visit MC Estimates:")
for state, val in list(value_first_visit_mc.items())[:5]:
    print(f"{state} → {val:.4f}")

print("\nEvery-Visit MC Estimates:")
for state, val in list(value_every_visit_mc.items())[:5]:
    print(f"{state} → {val:.4f}")


First-Visit MC Estimates:
(17, 10, False) → -0.6291
(20, 3, False) → -0.1427
(15, 10, False) → -0.6526
(20, 4, False) → -0.1407
(19, 4, False) → -0.2237

Every-Visit MC Estimates:
(14, 10, False) → -0.6453
(13, 7, False) → -0.5405
(13, 10, False) → -0.6211
(10, 10, False) → -0.5186
(16, 7, False) → -0.5645
